# Clustering Crypto

In [62]:
# Initial imports
import requests
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [63]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"


In [64]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
response_data = requests.get(url).json()
coin_url = pd.DataFrame(response_data)
coin_url.head()


,Response,Message,Data,RateLimit,HasWarning,Type
42,Success,Coin list succesfully returned!,"{'Id': '4321', 'Url': '/coins/42/overview', 'I...",NaN,False,100
300,Success,Coin list succesfully returned!,"{'Id': '749869', 'Url': '/coins/300/overview',...",NaN,False,100
365,Success,Coin list succesfully returned!,"{'Id': '33639', 'Url': '/coins/365/overview', ...",NaN,False,100
404,Success,Coin list succesfully returned!,"{'Id': '21227', 'Url': '/coins/404/overview', ...",NaN,False,100
433,Success,Coin list succesfully returned!,"{'Id': '926547', 'Url': '/coins/433/overview',...",NaN,False,100


In [65]:
# Alternatively, use the provided csv file:
file_path = Path('Resources/crypto_data.csv')
crypto_df = pd.read_csv(file_path)
crypto_df.head()

# Create a DataFrame

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [66]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.copy()
crypto_df.set_index('Unnamed: 0', inplace=True)

In [67]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [68]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading']== True]
crypto_df.shape

(1144, 6)

In [69]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df['Algorithm'] != "N/A"]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610


In [70]:
# Remove the "IsTrading" column
crypto_df.drop(['IsTrading'], axis=1, inplace=True)


In [71]:
# Remove rows with at least 1 null value
crypto_df.dropna(axis=0, how="any", inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [72]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.loc[crypto_df.TotalCoinsMined != 0]

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [73]:
# Drop rows where there are 'N/A' text values
crypto_df.dropna(axis=0,inplace=True, how='any')
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [74]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(crypto_df['CoinName'],index= crypto_df.index)

In [75]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns=['CoinName'], axis=1)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [76]:
coins_name.dropna()

,CoinName
Unnamed: 0,
42,42 Coin
404,404Coin
808,808
1337,EliteCoin
BTC,Bitcoin
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex


In [77]:
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11
BTC,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [78]:
# Create dummy variables for text features
X = pd.get_dummies(data=crypto_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
808,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,3.14159E+11,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)


### Reducing Dimensions Using PCA

In [85]:
# Use PCA to reduce dimensions to 3 principal components
n_comp = 3
pca = PCA(n_components=n_comp)
principal_comp= pca.fit_transform(X_scaled)
principal_comp

array([[-0.26941486,  1.19806169, -0.34676243],
       [-0.25501889,  1.19619642, -0.34729979],
       [-0.17020898,  0.8242362 , -0.19195187],
       ...,
       [-0.1129722 , -2.2997056 ,  0.17522572],
       [-0.17126715, -2.01031527,  0.22355296],
       [-0.13311143,  0.74008915, -0.16435688]])

In [86]:
# Create a DataFrame with the principal components data
col_names = [f"PC {i}" for i in range(1, n_comp + 1)]
pcs_df = pd.DataFrame(principal_comp, columns=col_names, index=crypto_df.index)
print(pcs_df.shape)
pcs_df.head(10)

(685, 3)


,PC 1,PC 2,PC 3
Unnamed: 0,,,
42,-0.269415,1.198062,-0.346762
404,-0.255019,1.196196,-0.347300
808,-0.170209,0.824236,-0.191952
1337,0.294242,1.968045,-0.403552
BTC,-0.212319,-1.453638,0.132870
ETH,-0.265515,-2.086154,0.238332
LTC,-0.310909,-1.079892,-0.021963
DASH,-0.203431,1.418743,-0.350728
XMR,-0.227840,-2.223961,0.231308


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [88]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k: 
    km = KMeans(n_clusters=i, random_state=8)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

c:\Users\wadee\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  f"KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [89]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=8)
model.fit(pcs_df)
pred_model = model.predict(pcs_df)

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pred_df = pd.concat([crypto_df, pcs_df], axis=1, sort= False)
pred_df['CoinName']= coins_name['CoinName']
pred_df['Clusters']= model.labels_
pred_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Clusters
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.269415,1.198062,-0.346762,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.255019,1.196196,-0.347300,404Coin,1
808,SHA-256,PoW/PoS,0.000000e+00,0,-0.170209,0.824236,-0.191952,808,1
1337,X13,PoW/PoS,2.927942e+10,3.14159E+11,0.294242,1.968045,-0.403552,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.212319,-1.453638,0.132870,Bitcoin,0


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [92]:
# Scale data to create the scatter plot
mm_scaler = MinMaxScaler()
plot_data = mm_scaler.fit_transform(
    pred_df[["TotalCoinSupply", "TotalCoinsMined"]]
)
plot_df = pd.DataFrame(
    plot_data, columns=["TotalCoinSupply", "TotalCoinsMined"], index=pred_df.index
)
plot_df["CoinName"] = pred_df["CoinName"]
plot_df["Clusters"] = pred_df["Clusters"]
plot_df.head()

,TotalCoinSupply,TotalCoinsMined,CoinName,Clusters
Unnamed: 0,,,,
42,4.553650e-16,0.005942,42 Coin,1
404,5.767957e-09,0.007002,404Coin,1
808,0.000000e+00,0.005942,808,1
1337,3.406119e-06,0.035342,EliteCoin,1
BTC,2.276825e-10,0.005960,Bitcoin,0


In [93]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", hover_cols=['CoinName'], by="Clusters")

:NdOverlay   [Clusters]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [94]:
# Table with tradable cryptos
pred_df[
    [
        "CoinName",
        "Algorithm",
        "ProofType",
        "TotalCoinSupply",
        "TotalCoinsMined",
        "Clusters",
    ]
].hvplot.table()

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Clusters]

In [96]:
# Print the total number of tradable cryptocurrencies
print(f'There are {pred_df.shape[0]} cryptocurrencies to trade.')

There are 685 cryptocurrencies to trade.
